In [1]:
import os
import nibabel as nib
import numpy as np
from nipype.interfaces.ants import N4BiasFieldCorrection
# import matplotlib.pyplot as plt
from scipy.ndimage.morphology import binary_dilation, binary_erosion
from copy import deepcopy

#from nipy import labs
import multiprocessing
from joblib import Parallel, delayed
from scipy.stats import iqr
import glob
import warnings # for file system warnings


C:\Users\ncwang\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import file_structure
(origdir,basedir,imagedir,normdir,savedir,train_dir,val_dir,test_dir) = file_structure.file_dirs()
os.chdir(imagedir)

patients = []
for root, dirs, files in os.walk('.'):
    for dn in dirs:
        patients.append(os.path.join(root,dn))
#normalized_dir = '/rsrch1/bcb/Imaging-Genomics_SIBL/DONNIE_KIM/Brachy_deep_learning/IDH_Prediction/data/normalized_data/'
os.chdir(origdir)

def normalize(p):

    print(p, patients[p])
    patient_dir = os.path.abspath(os.path.join(imagedir,patients[p]))
    patient_norm_dir = os.path.abspath(os.path.join(normdir,patients[p]))
    
    print(patient_norm_dir)
    #os.chdir(patient_dir)
    #print(patient_dir)
    
    # wildcard search
    #segTest = os.path.join(patient_dir,'*seg.nii.gz')
    #segTest = os.path.join(patient_dir,'*GlistrBoost_ManuallyCorrected.nii.gz')
    segTest = os.path.join(patient_dir,'*truth.nii.gz')
    segFiles = glob.glob(segTest)
    #normalize image intensity values relative to normal brain
    if not segFiles: # no files found
        print(segTest)
        segTest = os.path.join(patient_dir,'*GlistrBoost.nii.gz')
        segFiles = glob.glob(segTest)
        if not segFiles: # no files found
            print(segTest)
            return # require the truth file to exist

    if (len(segFiles) > 1): # glob is unsorted... throw a warning if multiple files found, pick the first one
        warnings.warn('Multiple files found, %s used' % (segFiles[0]))
        
    truth = nib.load(segFiles[0]).get_data()
    if not os.path.exists(patient_norm_dir):
        os.makedirs(patient_norm_dir)
    idx_mask = np.where(truth==0)
    out_name = os.path.abspath(os.path.join(patient_norm_dir,'truth.npy'))
    np.save(out_name, truth)
    
    # wildcard search
    flairTest = os.path.join(patient_dir,'*flair.nii.gz')
    flairFiles = glob.glob(flairTest)
    if flairTest: # file(s) found
        if (len(flairFiles) > 1): # glob is unsorted... throw a warning if multiple files found, pick the first one
            warnings.warn('Multiple files found, %s used' % (flairFiles[0]))
        FLAIR_ss = np.float64(np.round(nib.load(flairFiles[0]).get_data())) # try to save the flair version
        idx_nz = np.nonzero(FLAIR_ss[idx_mask])
        median = np.median(FLAIR_ss[idx_mask][idx_nz])
        curr_iqr = iqr(FLAIR_ss[idx_mask][idx_nz])
        FLAIR_normssn4 = deepcopy(FLAIR_ss)
        FLAIR_normssn4[np.nonzero(FLAIR_ss)] = (FLAIR_normssn4[np.nonzero(FLAIR_ss)]-median)/curr_iqr
        out_name = os.path.abspath(os.path.join(patient_norm_dir,'FLAIR_normssn4.npy'))
        np.save(out_name,FLAIR_normssn4)
    
    # wildcard search
    t2Test = os.path.join(patient_dir,'*t2.nii.gz')
    t2Files = glob.glob(t2Test)
    if t2Files: # file(s) found
        if (len(t2Files) > 1): # glob is unsorted... throw a warning if multiple files found, pick the first one
            warnings.warn('Multiple files found, %s used' % (t2Files[0]))
        T2_ss = np.float64(np.round(nib.load(t2Files[0]).get_data()))# try to save the t2 version
        idx_nz = np.nonzero(T2_ss[idx_mask])
        median = np.median(T2_ss[idx_mask][idx_nz])
        curr_iqr = iqr(T2_ss[idx_mask][idx_nz])
        T2_normssn4 = deepcopy(T2_ss)
        T2_normssn4[np.nonzero(T2_ss)] = (T2_normssn4[np.nonzero(T2_ss)]-median)/curr_iqr
        out_name = os.path.abspath(os.path.join(patient_norm_dir,'T2_normssn4.npy'))
        np.save(out_name,T2_normssn4)

    # wildcard search
    t1Test = os.path.join(patient_dir,'*t1.nii.gz')
    t1Files = glob.glob(t1Test)
    if t1Files: # file(s) found
        if (len(t1Files) > 1): # glob is unsorted... throw a warning if multiple files found, pick the first one
            warnings.warn('Multiple files found, %s used' % (t1Files[0]))
        T1_ss = np.float64(np.round(nib.load(t1Files[0]).get_data()))# try to save the t1 version
        idx_nz = np.nonzero(T1_ss[idx_mask])
        median = np.median(T1_ss[idx_mask][idx_nz])
        curr_iqr = iqr(T1_ss[idx_mask][idx_nz])
        T1_normssn4 = deepcopy(T1_ss)
        T1_normssn4[np.nonzero(T1_ss)] = (T1_normssn4[np.nonzero(T1_ss)]-median)/curr_iqr   
        out_name = os.path.abspath(os.path.join(patient_norm_dir,'T1_normssn4.npy'))
        np.save(out_name,T1_normssn4)
    
    # wildcard search
    t1GdTest = os.path.join(patient_dir,'*t1Gd.nii.gz')
    t1GdFiles = glob.glob(t1GdTest)
    if t1GdFiles: # file(s) found
        if (len(t1GdFiles) > 1): # glob is unsorted... throw a warning if multiple files found, pick the first one
            warnings.warn('Multiple files found, %s used' % (t1GdFiles[0]))
        T1post_ss = np.float64(np.round(nib.load(t1GdFiles[0]).get_data()))
        idx_nz = np.nonzero(T1post_ss[idx_mask])
        median = np.median(T1post_ss[idx_mask][idx_nz])
        curr_iqr = iqr(T1post_ss[idx_mask][idx_nz])
        T1post_normssn4 = deepcopy(T1post_ss)
        T1post_normssn4[np.nonzero(T1post_ss)] = (T1post_normssn4[np.nonzero(T1post_ss)]-median)/curr_iqr     
        out_name = os.path.abspath(os.path.join(patient_norm_dir,'T1post_normssn4.npy'))
        np.save(out_name,T1post_normssn4)         



In [3]:

#basedir = '/rsrch1/bcb/Imaging-Genomics_SIBL/DONNIE_KIM/Brachy_deep_learning/IDH_Prediction/data/image_data/'


num_cores = multiprocessing.cpu_count()
Parallel(n_jobs=num_cores)(delayed(normalize)(p) for p in range(len(patients)))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [4]:
p = 4

patient_dir = os.path.abspath(os.path.join(imagedir,patients[p]))
patient_norm_dir = os.path.abspath(os.path.join(normalized_dir,patients[p]))

print(patient_norm_dir)
#os.chdir(patient_dir)
#print(patient_dir)

# wildcard search
#segTest = os.path.join(patient_dir,'*seg.nii.gz')
#segTest = os.path.join(patient_dir,'*GlistrBoost_ManuallyCorrected.nii.gz')
segTest = os.path.join(patient_dir,'*truth.nii.gz')
segFiles = glob.glob(segTest)
#normalize image intensity values relative to normal brain
if not segFiles: # no files found
    print(segTest)
    segTest = os.path.join(patient_dir,'*GlistrBoost.nii.gz')
    segFiles = glob.glob(segTest)
    if not segFiles: # no files found
        print(segTest)

if (len(segFiles) > 1): # glob is unsorted... throw a warning if multiple files found, pick the first one
    warnings.warn('Multiple files found, %s used' % (segFiles[0]))

truth = nib.load(segFiles[0]).get_data()
if not os.path.exists(patient_norm_dir):
    os.makedirs(patient_norm_dir)
idx_mask = np.where(truth==0)
out_name = os.path.abspath(os.path.join(patient_norm_dir,'truth.npy'))
np.save(out_name, truth)


# wildcard search
t1Test = os.path.join(patient_dir,'*t1.nii.gz')
t1Files = glob.glob(t1Test)
if t1Files: # file(s) found
    if (len(t1Files) > 1): # glob is unsorted... throw a warning if multiple files found, pick the first one
        warnings.warn('Multiple files found, %s used' % (t1Files[0]))
    T1_ss = np.round(nib.load(t1Files[0]).get_data())# try to save the t1 version
    idx_nz = np.nonzero(T1_ss[idx_mask])
    median = np.median(T1_ss[idx_mask][idx_nz])
    curr_iqr = iqr(T1_ss[idx_mask][idx_nz])
    T1_normssn4 = deepcopy(T1_ss)
    T1_normssn4[np.nonzero(T1_ss)] = np.float(T1_normssn4[np.nonzero(T1_ss)]-median)/curr_iqr   
    out_name = os.path.abspath(os.path.join(patient_norm_dir,'T1_normssn4.npy'))
    np.save(out_name,T1_normssn4)

NameError: name 'normalized_dir' is not defined

In [ ]:
normalize(4)

In [ ]:
T1_ss = np.float64(np.round(nib.load(t1Files[0]).get_data()))
T1_normssn4 = deepcopy(T1_ss)
T1_normssn4[np.nonzero(T1_ss)] = (T1_normssn4[np.nonzero(T1_ss)]-median)/curr_iqr  
print(np.unique(T1_normssn4))